## Что такое Стриминг?

  
![Kafka](https://wiki.databurst.tech/docs/roadmap/distributed-systems-concepts/batch-and-stream-processing/batch-vs-streaming.png)



## Что такое Apache Kafka?
Apache Kafka — это распределённая система передачи сообщений (message broker), предназначенная для обработки и хранения потоковых данных.

- **Брокеры** — сервера, управляющие топиками и партициями
- **Топики** — логические каналы передачи данных
- **Партиции** — деление топика для масштабирования и параллелизма
- **Продюсеры** — публикуют данные
- **Консьюмеры** — читают данные
  
![Kafka](https://nuancesprog.ru/wp-content/uploads/2025/03/1QDHuTomu3e-A4I8WAKAO7Q.png)




In [ ]:
!pip install kafka-python 

In [ ]:

from kafka import KafkaProducer
import time

producer = KafkaProducer(bootstrap_servers='localhost:9092')
for i in range(5):
    msg = f"Hello {i}"
    producer.send('demo-topic', msg.encode())
    print("Sent:", msg)
    time.sleep(1)




In [ ]:
from kafka import KafkaConsumer

consumer = KafkaConsumer('demo-topic', bootstrap_servers='localhost:9092', auto_offset_reset='earliest', consumer_timeout_ms=1000)
for message in consumer:
    print("Received:", message.value.decode())

## Spark Structured Streaming: потоковая обработка данных из Kafka
- Используем Spark для чтения и обработки потоковых данных из Kafka.
- Основная идея: читать как DataFrame, применять SQL-подобные операции, записывать результаты.

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("StreamingApp") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.5") \
    .getOrCreate()

In [ ]:

# Чтение потока из Kafka
kafka_df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "enriched-titanic") \
    .option("startingOffsets", "earliest") \
    .load()

parsed_df = kafka_df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

query = parsed_df.writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

query.awaitTermination()


## Запись данных из Spark Structured Streaming обратно в Kafka

In [ ]:

kafka_df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "demo-topic") \
    .option("startingOffsets", "earliest") \
    .load()

parsed_df = kafka_df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

from pyspark.sql.functions import concat, lit, col

transformed_df = parsed_df.withColumn(
    "value",
    concat(lit("[processed] "), col("value"))
)

final_df = transformed_df.select(
    col("key").cast("binary"),
    col("value").cast("binary")
)

query = final_df.writeStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("topic", "demo-topic-2") \
    .option("checkpointLocation", "/tmp/checkpoints/processed") \
    .outputMode("append") \
    .start()

query.awaitTermination()



## Соединение потоковых и статичных данных в Spark


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, when, to_json, struct, from_json

spark = SparkSession.builder \
    .appName("TitanicKafkaStreaming") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0") \
    .getOrCreate()


titanic_df = spark.read.csv("titanic.csv", header=True, inferSchema=True)

titanic_to_kafka = titanic_df.selectExpr(
    "CAST(PassengerId AS STRING) AS key",
    "to_json(struct(*)) AS value"
)

titanic_to_kafka.write \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("topic", "raw-titanic") \
    .save()

print("Данные загружены в топик raw-titanic")




In [ ]:
schema = titanic_df.schema

raw_stream = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "raw-titanic") \
    .option("startingOffsets", "earliest") \
    .load()

parsed_stream = raw_stream.selectExpr("CAST(value AS STRING) as json_str") \
    .withColumn("data", from_json(col("json_str"), schema)) \
    .select("data.*")


static_df = spark.createDataFrame([
    (1, "First Class"),
    (2, "Second Class"),
    (3, "Third Class")
], ["Pclass", "ClassName"])


enriched_df = parsed_stream \
    .withColumn("is_adult", when(col("Age") >= 18, lit("yes")).otherwise(lit("no"))) \
    .join(static_df, on="Pclass", how="left")

pretty_df = enriched_df.select(
    "PassengerId", "Name", "Age", "Pclass", "ClassName", "is_adult", "Survived"
)


df_to_kafka = enriched_df \
    .withColumn("key", col("PassengerId").cast("string")) \
    .withColumn("value", to_json(struct([col(c) for c in enriched_df.columns])))


query_kafka = df_to_kafka.select("key", "value").writeStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("topic", "enriched-titanic") \
    .option("checkpointLocation", "/tmp/checkpoints/titanic-kafka") \
    .outputMode("append") \
    .start()

query_console = pretty_df.writeStream \
    .format("console") \
    .option("truncate", False) \
    .outputMode("append") \
    .start()

query_kafka.awaitTermination()
query_console.awaitTermination()

## Мониторинг и отладка потоковых приложений
- Spark UI (http://localhost:4040) для мониторинга задач.
- Checkpointing для восстановления состояния и обеспечения отказоустойчивости.

## Оконные функции

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr, to_json, struct, col

spark = SparkSession.builder \
    .appName("TitanicToKafkaWithEventTime") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0") \
    .getOrCreate()


titanic_df = spark.read.csv("titanic.csv", header=True, inferSchema=True)


titanic_with_time = titanic_df.withColumn(
    "event_time",
    expr("current_timestamp() + interval 5 seconds * PassengerId")
)

titanic_with_time.selectExpr(
    "CAST(PassengerId AS STRING) AS key",
    "to_json(struct(*)) AS value"
).write \
 .format("kafka") \
 .option("kafka.bootstrap.servers", "localhost:9092") \
 .option("topic", "raw-titanic") \
 .save()

print("Данные записаны в Kafka с event_time")


In [ ]:


from pyspark.sql.functions import from_json
from pyspark.sql.types import *
from pyspark.sql.functions import window

schema = titanic_with_time.schema

raw_stream = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "raw-titanic") \
    .option("startingOffsets", "earliest") \
    .load()

parsed_stream = raw_stream.selectExpr("CAST(value AS STRING) as json_str") \
    .withColumn("data", from_json(col("json_str"), schema)) \
    .select("data.*")



windowed_counts = parsed_stream \
    .groupBy(
        window(col("event_time"), "1 minute")
    ).count() \
    .orderBy("window")

query = windowed_counts.writeStream \
    .format("console") \
    .outputMode("complete") \
    .option("truncate", False) \
    .start()

query.awaitTermination()

## Визуализация

In [ ]:
!pip install matplotlib

In [ ]:
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, from_json
import time


spark = SparkSession.builder \
    .appName("TitanicStreamingBatchSave") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0") \
    .getOrCreate()


schema = spark.read.csv("titanic.csv", header=True, inferSchema=True).schema


raw_stream = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "raw-titanic") \
    .option("startingOffsets", "earliest") \
    .load()


parsed_stream = raw_stream.selectExpr("CAST(value AS STRING) as json_str") \
    .withColumn("data", from_json(col("json_str"), schema)) \
    .select("data.*")


enriched_stream = parsed_stream.withColumn("is_adult", when(col("Age") >= 18, "yes").otherwise("no"))


agg_stream = enriched_stream.groupBy("is_adult").count()


query = agg_stream.writeStream \
    .format("memory") \
    .queryName("agg_table") \
    .outputMode("complete") \
    .start()


query.awaitTermination(10)


query.stop()


result_df = spark.sql("SELECT * FROM agg_table")


pdf = result_df.toPandas()


plt.bar(pdf['is_adult'], pdf['count'])
plt.xlabel('is_adult')
plt.ylabel('Count')
plt.title('Количество взрослых и не взрослых пассажиров Titanic (batch из стрима)')
for i, v in enumerate(pdf['count']):
    plt.text(i, v + 1, str(v), ha='center')
plt.show()


